The following is a google colab version first step is to load all the requirements


In [0]:
#from google.colab import files
#uploaded = files.upload()

Import all nessecary liraries and data to variables

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
import math
import xgboost
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
import datetime

In [0]:
train = pd.read_csv("Train.csv")
test = pd.read_csv("Test.csv")

Seperate Date time info of Train and test dataset. Because follow a peak at some specific that implies it is time dependent



In [0]:

train = train.astype(str).apply(lambda x: x.str.split(':').str[0])
train["time"] = train["date_time"].apply(lambda x: x.split(' ')[1])
train["date"] = train["date_time"].apply(lambda x: x.split(' ')[0])
yr_tr = train["date"].apply(lambda x: x.split('-')[0]).astype(int)
mn_tr = train["date"].apply(lambda x: x.split('-')[1]).astype(int)
dt_tr = train["date"].apply(lambda x: x.split('-')[2]).astype(int)
train["day"] = [datetime.date(x, y, z).strftime("%w") for z, y, x in zip(dt_tr, mn_tr, yr_tr)]
test_dati = test["date_time"]
test = test.astype(str).apply(lambda x: x.str.split(':').str[0])
test["time"] = test["date_time"].apply(lambda x: x.split(' ')[1])
test["date"] = test["date_time"].apply(lambda x: x.split(' ')[0])
yr_ts = test["date"].apply(lambda x: x.split('-')[0]).astype(int)
mn_ts = test["date"].apply(lambda x: x.split('-')[1]).astype(int)
dt_ts = test["date"].apply(lambda x: x.split('-')[2]).astype(int)
test["day"] = [datetime.date(x, y, z).strftime("%w") for z, y, x in zip(dt_ts, mn_ts, yr_ts)]
#view train data
train.iloc[5000:].head(5)

,date_time,is_holiday,air_pollution_index,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,weather_type,weather_description,traffic_volume,time,date,day
5000,2013-04-11 14,None,294,37,2,200,8,8,275.89,0.0,0.0,90,Clouds,overcast clouds,4562,14,2013-04-11,4
5001,2013-04-11 15,None,102,32,2,240,1,1,276.25,0.0,0.0,90,Haze,haze,4797,15,2013-04-11,4
5002,2013-04-11 16,None,264,28,2,310,6,6,276.18,0.0,0.0,90,Clouds,overcast clouds,5668,16,2013-04-11,4
5003,2013-04-11 17,None,171,28,2,320,7,7,276.29,0.0,0.0,90,Haze,haze,5029,17,2013-04-11,4
5004,2013-04-11 18,None,261,28,3,300,2,2,276.2,0.0,0.0,90,Rain,light rain,3803,18,2013-04-11,4


In [0]:
#view test data
test.head()

#Preprocessing

Seperate the input and output variables and clean some unnessecary collummns for train and test data

In [0]:
train_y = train.pop("traffic_volume")
train = train.drop(["weather_type", "weather_description", "dew_point", "date_time", "date"], axis=1)
#test_dati = test["date_time"]
test = test.drop(["weather_type", "weather_description", "dew_point", "date_time", "date"], axis=1)

Add holiday effect and normalize data

In [0]:
train.loc[train.is_holiday == 'None', 'is_holiday'] = 0
train.loc[train.is_holiday != 0, 'is_holiday'] = 1
test.loc[test.is_holiday == 'None', 'is_holiday'] = 0
test.loc[test.is_holiday != 0, 'is_holiday'] = 1
tr_col = train.columns
ts_col = test.columns
#train['air_pollution_index'] = 'None', 'is_holiday'] = 0
#test.loc[test.is_holiday != 0, 'is_holiday'] = 1
train = preprocessing.normalize(train)
test = preprocessing.normalize(test)
train = pd.DataFrame(data=train, columns=tr_col)
test = pd.DataFrame(data=test, columns=ts_col)


In [0]:

train.head()

In [0]:
test.head()

In [0]:
#x_train, x_test, y_train, y_test = train_test_split(train, train_y, test_size =0.1)
x_train = train
y_train = train_y

In [0]:
#Check Pearson Correlation
colormap = plt.cm.RdBu
plt.figure(figsize=(14,12))
plt.title('Pearson Correlation of Features', y=1.05, size=15)
sns.heatmap(train.astype(float).corr(),linewidths=0.1,vmax=1.0, 
            square=True, cmap=colormap, linecolor='white', annot=True)

In [0]:
'''knr = KNeighborsRegressor(n_neighbors=5)
dtr = tree.DecisionTreeRegressor(max_depth = 5)
r = Ridge(alpha=0.5)
#try support vector machine


knr.fit(x_train, y_train)
knr_tr = np.round(knr.predict(x_train))
print(knr_tr)
knr_st = knr.predict(x_test)
dtr.fit(x_train, y_train) 
dtr_tr = dtr.predict(x_train)
dtr_st = dtr.predict(x_test)
lr.fit(x_train, y_train)
lr_tr = np.round(lr.predict(x_train))
lr_st = lr.predict(x_test)


x_train = np.concatenate((knr_tr), (lr_tr))
x_test = np.concatenate(knr_st, lr_st)'''

lr = GridSearchCV(
        estimator=RandomForestRegressor(),
        param_grid={
            'max_depth': range(3,10),
            'n_estimators': (10, 50, 500, 1000),
        },
        cv=5, scoring='neg_mean_squared_error', verbose=0,n_jobs=-1)


model = lr.fit(x_train, y_train)


In [0]:
#plt.figure(figsize=(30,5))
#plt.scatter(train.columns.values, importance, s = np.pi*15)
best_params = model.best_params_
    
rfr = RandomForestRegressor(max_depth=best_params["max_depth"], n_estimators=best_params["n_estimators"], random_state=False, verbose=False)

In [0]:
test_res = (lr.predict(x_test))
#error = model.score(x_test, y_test)
error = cross_val_score(rfr, x_train, y_train, cv=10, scoring='neg_mean_absolute_error')
print(error)

In [0]:
result = list((lr.predict(test)))

In [0]:
results = pd.DataFrame({'date_time': test_dati,'traffic_volume': result})
results.head()

In [0]:
results.to_csv('predict.csv', index=False)

In [0]:
from google.colab import files
files.download('predict.csv')